In [5]:
from hyperopt import fmin, hp, tpe, SparkTrials, STATUS_OK
from hyperopt.early_stop import no_progress_loss

from hpsklearn import any_regressor, any_preprocessing
import sklearn.preprocessing
from sklearn.linear_model import LinearRegression

from lightgbm import LGBMRegressor
from xgboost.sklearn import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR


In [6]:
lag_options = [ [1], [1, 2], [1, 2, 3], [1, 2, 3, 4], [1, 2, 3, 4, 5], [1, 2, 3, 4, 5, 6], [1, 2, 3, 4, 5, 6, 7], [1, 2, 3, 4, 5, 6, 7, 8], [1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 25], [1, 2, 25, 26], [1, 2, 3, 25, 26, 27], [1, 2, 3, 4, 25, 26, 27, 28], [1, 2, 3, 4, 5, 25, 26, 27, 28, 29], [1, 2, 3, 4, 5, 6, 25, 26, 27, 28, 29, 30], [1, 2, 3, 4, 5, 6, 7, 25, 26, 27, 28, 29, 30, 31], [1, 2, 3, 4, 5, 6, 7, 8, 25, 26, 27, 28, 29, 30, 31, 32], [1, 2, 3, 4, 5, 6, 7, 8, 9, 25, 26, 27, 28, 29, 30, 31, 32, 33], [1, 25, 49], [1, 2, 25, 26, 49, 50], [1, 2, 3, 25, 26, 27, 49, 50, 51], [1, 2, 3, 4, 25, 26, 27, 28, 49, 50, 51, 52], [1, 2, 3, 4, 5, 25, 26, 27, 28, 29, 49, 50, 51, 52, 53], [1, 2, 3, 4, 5, 6, 25, 26, 27, 28, 29, 30, 49, 50, 51, 52, 53, 54], [1, 2, 3, 4, 5, 6, 7, 25, 26, 27, 28, 29, 30, 31, 49, 50, 51, 52, 53, 54, 55], [1, 2, 3, 4, 5, 6, 7, 8, 25, 26, 27, 28, 29, 30, 31, 32, 49, 50, 51, 52, 53, 54, 55, 56], [1, 2, 3, 4, 5, 6, 7, 8, 9, 25, 26, 27, 28, 29, 30, 31, 32, 33, 49, 50, 51, 52, 53, 54, 55, 56, 57]]

In [9]:
linreg = LinearRegression()
lgbm = LGBMRegressor()
# xgb = XGBRegressor()
catboost = CatBoostRegressor()
sgd = SGDRegressor()
kernelridge = KernelRidge()
elastic = ElasticNet()
bayesianridge = BayesianRidge()
# gradientboosting = GradientBoostingRegressor(n_estimators=100)
svr = SVR()

# all_regressors = [linreg, lgbm, xgb, catboost, sgd, kernelridge, elastic, bayesianridge, gradientboosting, svr]
all_regressors = [linreg, lgbm, catboost, sgd, kernelridge, elastic, bayesianridge, svr]

def get_params():
    search_space = {#"price_1": hp.uniform("price_1", -1, 1),
                    # "cap_learning_data": hp.quniform("cap_learning_data", 300, 800, 100),
                    # "update_learner_interval": hp.quniform("update_learner_interval", 1, 4, 1),
                    "cap_learning_data": hp.choice("cap_learning_data", [500]),
                    "update_learner_interval": hp.choice("update_learner_interval", [4]),
                    "load_lags": hp.choice("load_lags", lag_options),
                    "solar_lags": hp.choice("solar_lags", lag_options),
                    # "load_regressor": hp.choice("load_regressor", all_regressors),
                    # "solar_regressor": hp.choice("solar_regressor", all_regressors),

                    # "load_regressor": any_regressor("my_reg1"),
                    # "solar_regressor": any_regressor("my_reg2"),
                    # "load_transformer": any_preprocessing("my_pre1"),
                    # "solar_transformer": any_preprocessing("my_pre2")
                    }
    return search_space

In [ ]:
from traineval.evaluation.tune_evaluation import evaluate

best_params = fmin(
    fn=evaluate,
    space=get_params(),
    algo=tpe.suggest,  # NOTE: You cannot use atpe.suggest with SparkTrials, then use tpe.suggest
    max_evals=8000,
    trials=SparkTrials(),
    early_stop_fn=no_progress_loss(400)
)
best_params

Because the requested parallelism was None or a non-positive value, parallelism will be set to (24), which is Spark's default parallelism (24), or 1, whichever is greater. We recommend setting parallelism explicitly to a positive value because the total of Spark task slots is subject to cluster sizing.
hyperopt-spark WARNING Because the requested parallelism was None or a non-positive value, parallelism will be set to (24), which is Spark's default parallelism (24), or 1, whichever is greater. We recommend setting parallelism explicitly to a positive value because the total of Spark task slots is subject to cluster sizing.


  0%|          | 0/8000 [00:00<?, ?trial/s, best loss=?]

hyperopt.tpe INFO  build_posterior_wrapper took 0.015631 seconds
hyperopt.tpe INFO  TPE using 0 trials


  0%|          | 0/8000 [00:01<?, ?trial/s, best loss=?]


py4j.clientserver INFO  Closing down clientserver connection
